# Import

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoint using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Preprocess Data and Create Labels and Features

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
actions = np.array(['fine','thanks','hi'])
no_sequences = 30
sequence_length = 60
DATA_PATH = os.path.join('M:\Work\MIT\dataset') 

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
label_map

{'fine': 0, 'thanks': 1, 'hi': 2}

In [10]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
np.array(sequences).shape

(90, 60, 1662)

In [12]:
np.array(labels).shape

(90,)

In [13]:
X = np.array(sequences)

In [14]:
X.shape

(90, 60, 1662)

In [15]:
y = to_categorical(labels).astype(int)

In [16]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
y_test.shape

(5, 3)

# Build and Train LSTM Neural Network

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
og_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir="M:\Work\MIT\logs")

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 107ms/step - loss: 196.7409 - categorical_accuracy: 0.2824
Epoch 2/2000
3/3 [==============================] - 0s 115ms/step - loss: 89.2128 - categorical_accuracy: 0.3882
Epoch 3/2000
3/3 [==============================] - 0s 116ms/step - loss: 68.8738 - categorical_accuracy: 0.3294
Epoch 4/2000
3/3 [==============================] - 0s 114ms/step - loss: 44.3606 - categorical_accuracy: 0.4000
Epoch 5/2000
3/3 [==============================] - 0s 113ms/step - loss: 57.5382 - categorical_accuracy: 0.4235
Epoch 6/2000
3/3 [==============================] - 0s 113ms/step - loss: 166.8962 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 110ms/step - loss: 477.4043 - categorical_accuracy: 0.3765
Epoch 8/2000
3/3 [==============================] - 0s 116ms/step - loss: 532.9097 - categorical_accuracy: 0.3647
Epoch 9/2000
3/3 [==============================] - 0s 121ms/step - loss: 667.7393 - categor

3/3 [==============================] - 0s 130ms/step - loss: 1225.7706 - categorical_accuracy: 0.3294
Epoch 73/2000
3/3 [==============================] - 0s 129ms/step - loss: 1063.5662 - categorical_accuracy: 0.3412
Epoch 74/2000
3/3 [==============================] - 0s 125ms/step - loss: 1181.6696 - categorical_accuracy: 0.3059
Epoch 75/2000
3/3 [==============================] - 0s 130ms/step - loss: 924.8198 - categorical_accuracy: 0.3176
Epoch 76/2000
3/3 [==============================] - 0s 127ms/step - loss: 1119.5193 - categorical_accuracy: 0.3059
Epoch 77/2000
3/3 [==============================] - 0s 122ms/step - loss: 846.9915 - categorical_accuracy: 0.3882
Epoch 78/2000
3/3 [==============================] - 0s 119ms/step - loss: 1351.8964 - categorical_accuracy: 0.2235
Epoch 79/2000
3/3 [==============================] - 0s 120ms/step - loss: 432.3146 - categorical_accuracy: 0.2118
Epoch 80/2000
3/3 [==============================] - 0s 125ms/step - loss: 385.2572 - cat

3/3 [==============================] - 1s 163ms/step - loss: 124.4865 - categorical_accuracy: 0.3882
Epoch 214/2000
3/3 [==============================] - 1s 183ms/step - loss: 154.6603 - categorical_accuracy: 0.3412
Epoch 215/2000
3/3 [==============================] - 1s 181ms/step - loss: 176.4401 - categorical_accuracy: 0.3647
Epoch 216/2000
3/3 [==============================] - 1s 171ms/step - loss: 191.7943 - categorical_accuracy: 0.5176
Epoch 217/2000
3/3 [==============================] - 1s 162ms/step - loss: 269.6013 - categorical_accuracy: 0.3412
Epoch 218/2000
3/3 [==============================] - 1s 175ms/step - loss: 298.5564 - categorical_accuracy: 0.3882
Epoch 219/2000
3/3 [==============================] - 0s 160ms/step - loss: 208.8081 - categorical_accuracy: 0.3412
Epoch 220/2000
3/3 [==============================] - 1s 167ms/step - loss: 141.0038 - categorical_accuracy: 0.5059
Epoch 221/2000
3/3 [==============================] - 1s 183ms/step - loss: 123.5907 - 

3/3 [==============================] - 1s 160ms/step - loss: 21.4165 - categorical_accuracy: 0.4235
Epoch 355/2000
3/3 [==============================] - 1s 173ms/step - loss: 17.6804 - categorical_accuracy: 0.4941
Epoch 356/2000
3/3 [==============================] - 1s 176ms/step - loss: 26.2481 - categorical_accuracy: 0.4118
Epoch 357/2000
3/3 [==============================] - 1s 167ms/step - loss: 22.3144 - categorical_accuracy: 0.3294
Epoch 358/2000
3/3 [==============================] - 1s 165ms/step - loss: 14.6695 - categorical_accuracy: 0.4588
Epoch 359/2000
3/3 [==============================] - 1s 176ms/step - loss: 17.3151 - categorical_accuracy: 0.4235
Epoch 360/2000
3/3 [==============================] - 1s 163ms/step - loss: 28.8685 - categorical_accuracy: 0.3765
Epoch 361/2000
3/3 [==============================] - 0s 158ms/step - loss: 26.0455 - categorical_accuracy: 0.4235
Epoch 362/2000
3/3 [==============================] - 1s 160ms/step - loss: 32.4515 - categoric

3/3 [==============================] - 1s 171ms/step - loss: 99.4841 - categorical_accuracy: 0.3647
Epoch 497/2000
3/3 [==============================] - 1s 180ms/step - loss: 111.3427 - categorical_accuracy: 0.2706
Epoch 498/2000
3/3 [==============================] - 1s 163ms/step - loss: 85.1308 - categorical_accuracy: 0.3412
Epoch 499/2000
3/3 [==============================] - 1s 166ms/step - loss: 80.8520 - categorical_accuracy: 0.3647
Epoch 500/2000
3/3 [==============================] - 1s 181ms/step - loss: 46.9574 - categorical_accuracy: 0.4235
Epoch 501/2000
3/3 [==============================] - 1s 171ms/step - loss: 37.9944 - categorical_accuracy: 0.4353
Epoch 502/2000
3/3 [==============================] - 1s 167ms/step - loss: 86.0631 - categorical_accuracy: 0.3882
Epoch 503/2000
3/3 [==============================] - 1s 163ms/step - loss: 46.2452 - categorical_accuracy: 0.4235
Epoch 504/2000
3/3 [==============================] - 1s 165ms/step - loss: 87.4106 - categori

...

3/3 [==============================] - 1s 188ms/step - loss: 11.4421 - categorical_accuracy: 0.4353
Epoch 1632/2000
3/3 [==============================] - 1s 184ms/step - loss: 10.2113 - categorical_accuracy: 0.4471
Epoch 1633/2000
3/3 [==============================] - 1s 182ms/step - loss: 8.6483 - categorical_accuracy: 0.3882
Epoch 1634/2000
3/3 [==============================] - 1s 183ms/step - loss: 18.2641 - categorical_accuracy: 0.2941
Epoch 1635/2000
3/3 [==============================] - 1s 187ms/step - loss: 11.4975 - categorical_accuracy: 0.3294
Epoch 1636/2000
3/3 [==============================] - 1s 192ms/step - loss: 12.5755 - categorical_accuracy: 0.4941
Epoch 1637/2000
3/3 [==============================] - 1s 176ms/step - loss: 10.4646 - categorical_accuracy: 0.5176
Epoch 1638/2000
3/3 [==============================] - 1s 184ms/step - loss: 10.8166 - categorical_accuracy: 0.4235
Epoch 1639/2000
3/3 [==============================] - 1s 180ms/step - loss: 20.3336 - ca

Epoch 1702/2000
3/3 [==============================] - 1s 198ms/step - loss: 19.8506 - categorical_accuracy: 0.3882
Epoch 1703/2000
3/3 [==============================] - 1s 197ms/step - loss: 9.9205 - categorical_accuracy: 0.4824
Epoch 1704/2000
3/3 [==============================] - 1s 194ms/step - loss: 9.8970 - categorical_accuracy: 0.4941
Epoch 1705/2000
3/3 [==============================] - 1s 192ms/step - loss: 7.9479 - categorical_accuracy: 0.4118
Epoch 1706/2000
3/3 [==============================] - 1s 191ms/step - loss: 5.0601 - categorical_accuracy: 0.4471
Epoch 1707/2000
3/3 [==============================] - 1s 189ms/step - loss: 5.3994 - categorical_accuracy: 0.4706
Epoch 1708/2000
3/3 [==============================] - 1s 192ms/step - loss: 5.6808 - categorical_accuracy: 0.4706
Epoch 1709/2000
3/3 [==============================] - 1s 195ms/step - loss: 7.4689 - categorical_accuracy: 0.4941
Epoch 1710/2000
3/3 [==============================] - 1s 202ms/step - loss: 17

3/3 [==============================] - 1s 190ms/step - loss: 13.0767 - categorical_accuracy: 0.4706
Epoch 1844/2000
3/3 [==============================] - 1s 191ms/step - loss: 9.5915 - categorical_accuracy: 0.4588
Epoch 1845/2000
3/3 [==============================] - 1s 199ms/step - loss: 10.1165 - categorical_accuracy: 0.3765
Epoch 1846/2000
3/3 [==============================] - 1s 210ms/step - loss: 10.8547 - categorical_accuracy: 0.4353
Epoch 1847/2000
3/3 [==============================] - 1s 194ms/step - loss: 13.7220 - categorical_accuracy: 0.3882
Epoch 1848/2000
3/3 [==============================] - 1s 200ms/step - loss: 7.9293 - categorical_accuracy: 0.4941
Epoch 1849/2000
3/3 [==============================] - 1s 192ms/step - loss: 6.6757 - categorical_accuracy: 0.5059
Epoch 1850/2000
3/3 [==============================] - 1s 197ms/step - loss: 6.4795 - categorical_accuracy: 0.4353
Epoch 1851/2000
3/3 [==============================] - 1s 204ms/step - loss: 4.0784 - catego

Epoch 1914/2000
3/3 [==============================] - 1s 190ms/step - loss: 6.9841 - categorical_accuracy: 0.4824
Epoch 1915/2000
3/3 [==============================] - 1s 192ms/step - loss: 7.4960 - categorical_accuracy: 0.5647
Epoch 1916/2000
3/3 [==============================] - 1s 197ms/step - loss: 8.5349 - categorical_accuracy: 0.3412
Epoch 1917/2000
3/3 [==============================] - 1s 220ms/step - loss: 11.9467 - categorical_accuracy: 0.5059
Epoch 1918/2000
3/3 [==============================] - 1s 211ms/step - loss: 6.5560 - categorical_accuracy: 0.4824
Epoch 1919/2000
3/3 [==============================] - 1s 195ms/step - loss: 6.3628 - categorical_accuracy: 0.5176
Epoch 1920/2000
3/3 [==============================] - 1s 208ms/step - loss: 10.7241 - categorical_accuracy: 0.3882
Epoch 1921/2000
3/3 [==============================] - 1s 205ms/step - loss: 8.7553 - categorical_accuracy: 0.5059
Epoch 1922/2000
3/3 [==============================] - 1s 194ms/step - loss: 1

Epoch 1985/2000
3/3 [==============================] - 1s 206ms/step - loss: 17.3822 - categorical_accuracy: 0.3412
Epoch 1986/2000
3/3 [==============================] - 1s 208ms/step - loss: 19.4361 - categorical_accuracy: 0.5412
Epoch 1987/2000
3/3 [==============================] - 1s 208ms/step - loss: 16.3173 - categorical_accuracy: 0.5176
Epoch 1988/2000
3/3 [==============================] - 1s 209ms/step - loss: 12.2525 - categorical_accuracy: 0.3765
Epoch 1989/2000
3/3 [==============================] - 1s 209ms/step - loss: 17.3136 - categorical_accuracy: 0.4588
Epoch 1990/2000
3/3 [==============================] - 1s 202ms/step - loss: 10.4576 - categorical_accuracy: 0.4118
Epoch 1991/2000
3/3 [==============================] - 1s 202ms/step - loss: 12.8380 - categorical_accuracy: 0.5412
Epoch 1992/2000
3/3 [==============================] - 1s 205ms/step - loss: 9.9692 - categorical_accuracy: 0.5294
Epoch 1993/2000
3/3 [==============================] - 1s 196ms/step - lo

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 60, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Make Predictions

In [25]:
res = model.predict(X_test)

1/1 [==============================] - 0s 295ms/step


In [26]:
actions[np.argmax(res[4])]

'fine'

In [27]:
actions[np.argmax(y_test[4])]

'fine'

# Save Weights

In [31]:
model.save('action.h5')

NameError: name 'model' is not defined

In [29]:
del model

In [34]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.load_weights('action.h5')

# Evaluation using Confusion Matrix and Accuracy

In [35]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [36]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 246ms/step


In [37]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [38]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 2],
        [0, 2]],

       [[3, 0],
        [1, 1]],

       [[4, 0],
        [1, 0]]], dtype=int64)

In [39]:
accuracy_score(ytrue, yhat)

0.6

# Test in Real Time

In [41]:
from scipy import stats

In [42]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [44]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.soluti

...

1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 15ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
hi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution

1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
hi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
hi
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
fine
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
fine
<class 'mediapipe.python.soluti